<a href="https://drawdown.org"><img style="float: left;" src="data/images/ProjectDrawdown.png"></a>
<br clear=left>&nbsp;<br/>

**[The World’s Leading Resource for Climate Solutions](https://drawdown.org)**

Our mission is to help the world reach “Drawdown”—the point in the future when levels of greenhouse gases in the atmosphere stop climbing and start to steadily decline, thereby stopping catastrophic climate change—as quickly, safely, and equitably as possible

+ There is a **<a href="https://youtu.be/RAQSnhm_tS4" target="_blank">video demonstrating</a>** how to use the system.  
+ If you run into trouble or need assistance, the **<a href="https://solutions.geekhold.com/services/discourse/" target="_blank">Community Forum</a>** can help.  

In [1]:
import ui.charts
(j, tabs) = ui.charts.get_ui()
tabs